In [1]:
import numpy as np
import torch
from scipy.io import loadmat
import pandas as pd

In [2]:
def SkewSymmMatr(a : np.array) -> np.array:
    Ax= np.array(
        [[0.0,-a[2],a[1]],
        [a[2],0.0,-a[0]],
        [-a[1],a[0],0.0]])
    return Ax

In [3]:
def cross(a : np.array ,b : np.array) -> np.array:
    axb = SkewSymmMatr(a) @ b
    return axb

In [4]:
def twoPiBound(heading : float) -> float:
    if heading < 0:
        heading = heading + 2.0 * np.pi

    return heading

In [5]:
def LatLonAlt2XYZ(Lat : float,Lon : float,Alt : float) -> tuple:
    a_e = 6378137
    ecc = 0.081819190842622
    ksi = 1.0 / np.sqrt(1 - ecc * ecc * np.sin(Lat) * np.sin(Lat))
    X = (a_e * ksi + Alt) * np.cos(Lat) * np.cos(Lon)
    Y = (a_e * ksi + Alt) * np.cos(Lat) * np.sin(Lon)
    Z = (a_e * ksi * (1 - ecc * ecc) + Alt) * np.sin(Lat)
    return X, Y, Z

In [6]:
def CalcGravity(X:float,Y:float,Z:float)->np.array:
    t2 = X ** 2.0
    t3 = Y ** 2.0
    t4 = Z ** 2.0
    t5 = t2 + t3 + t4
    t6 = 1.0 / t5 ** (3.0 / 2.0)
    t7 = 1.0 / t5 ** (5.0 / 2.0)
    t8 = 1.0 / t5 ** (7.0 / 2.0)
    g = np.array([
       X * 5.3174941173225e-9-X * t6 * 3.9860142624e14-X * t7 * 2.633274580483414e25 + X * t4 * t8 * 1.316637290241707e26,
       Y * 5.3174941173225e-9 - Y * t6 * 3.9860142624e14 - Y * t7 * 2.633274580483414e25 + Y * t4 * t8 * 1.316637290241707e26,
       Z * t6 * -3.9860142624e14 - Z * t7 * 7.899823741450242e25 + Z * t4 * t8 * 1.316637290241707e26])
    return g

In [7]:
def CalcDCMnue_align(Pos : list,fb : np.array,om_ib : np.array) -> np.array:
    #nue->ned
    fb =np.array([fb[0],fb[2],-fb[1]])
    om_ib = np.array([om_ib[0],om_ib[2],-om_ib[1]])

    om_ie = 7.292115e-5

    cosB = np.cos(Pos[0])
    sinB = np.sin(Pos[0])
    cosL = np.cos(Pos[1])
    sinL = np.sin(Pos[1])

    Lat = Pos[0]

    (X,Y,Z) = LatLonAlt2XYZ(Pos[0],Pos[1],Pos[2])
    gravity = CalcGravity(X,Y,Z)

    gt = -cosL*sinB*gravity[0] - sinL*sinB*gravity[1] + cosB*gravity[2]
    gn = -(cosL*cosB*gravity[0] + sinL*cosB*gravity[1] + sinB*gravity[2])

    b1 = -fb
    b2 = om_ib
    b3 = cross(b1, b2)

    Mb = np.array([b1,b2,b3])

    Mn = np.array([[np.sin(Lat) / (gn * np.cos(Lat) + gt * np.sin(Lat)),gn / (gn * om_ie * np.cos(Lat) + gt * om_ie * np.sin(Lat)),0 ],
                  [0,0, 1 / (gn * om_ie * np.cos(Lat) + gt * om_ie * np.sin(Lat))],
                  [np.cos(Lat) / (gn * np.cos(Lat)+ gt * np.sin(Lat)),  -gt / (gn * om_ie * np.cos(Lat) + gt * om_ie * np.sin(Lat)), 0 ]] )

    R_nb = Mn @ Mb

    DCM = R_nb.T

    return DCM

In [8]:
def quat(H, gamma, theta):
  q0 = np.cos(H/2)*np.cos(theta/2)*np.cos(gamma/2)+np.sin(H/2)*np.sin(theta/2)*np.sin(gamma/2) 
  q1 = np.cos(H/2)*np.sin(theta/2)*np.cos(gamma/2)+np.sin(H/2)*np.cos(theta/2)*np.sin(gamma/2) #+
  q2 = np.cos(H/2)*np.cos(theta/2)*np.sin(gamma/2)-np.sin(H/2)*np.sin(theta/2)*np.cos(gamma/2) #-
  q3 = np.cos(H/2)*np.sin(theta/2)*np.sin(gamma/2)-np.sin(H/2)*np.cos(theta/2)*np.cos(gamma/2) #-
  Q = np.array([q0, q1, q2, q3])
  return Q

In [9]:
# МНК BODY-> Navigation (ENUp)
def DCM_bn(heading : float, pitch : float, roll : float) -> np.array:
    sz = np.sin(-heading)
    cz = np.cos(-heading)
    sy = np.sin(roll)
    cy = np.cos(roll)
    sx = np.sin(pitch)
    cx = np.cos(pitch)

    Cx = np.array([[1, 0, 0],[0, cx, sx],[0, -sx, cx]],dtype = np.float64)
    Cy = np.array([[cy, 0, -sy],[0, 1, 0],[sy, 0, cy]],dtype = np.float64)
    Cz = np.array([[cz, sz, 0],[-sz, cz, 0],[0, 0, 1]],dtype = np.float64)

    Cnb = Cy @ Cx @ Cz

    return Cnb.T

In [10]:
def quat2DCM(Q2:np.array):
  c11 = Q2[0]**2 + Q2[1]**2 - Q2[2]**2 - Q2[3]**2
  c12 = 2*(Q2[1]*Q2[2] - Q2[0]*Q2[3])
  c13 = 2*(Q2[1]*Q2[3] + Q2[0]*Q2[2])
  c21 = 2*(Q2[1]*Q2[2] + Q2[0]*Q2[3])
  c22 = Q2[0]**2 - Q2[1]**2 + Q2[2]**2 - Q2[3]**2
  c23 = 2*(Q2[2]*Q2[3] - Q2[0]*Q2[1])
  c31 = 2*(Q2[1]*Q2[3] - Q2[0]*Q2[2])
  c32 = 2*(Q2[2]*Q2[3] + Q2[0]*Q2[1])
  c33 = Q2[0]**2 - Q2[1]**2 - Q2[2]**2 + Q2[3]**2
  C_bLL = np.array([[c11, c12, c13], [c21, c22, c23], [c31, c32, c33]])
  return C_bLL

In [22]:
mat = loadmat('data/nav.binB_03.mat')
data_df = pd.DataFrame(mat['UAV'])
data = data_df.to_numpy().T
data = np.delete(data, 0, 1)
data = data[405500:,:]
n = data.shape[0]
acc = data[:,9:12]
gyro = data[:,12:15]

GPSQuality = data[:,23]
GPS = data[:,17:23]
GPS[:,0:2] = GPS[:,0:2] * np.pi / 180

StatusMS = data[:, 33]
MagnSens = data[:, 30:33] * 10
MagnSens[:,1:3] = -1 * MagnSens[:,1:3]

dt = 1 / 100
deg_to_rad = np.pi/180
rad_to_deg = 1/deg_to_rad

In [12]:
def threeaxisrot(r11, r12, r21, r31, r32):
    r1 = np.arctan2(r11,r12)
    r2 = np.arcsin(r21)
    r3 = np.arctan2(r31,r32)
    return r1,r2,r3

def EulerFromDCMnue(dcm):
    (r1, r2, r3) = threeaxisrot(dcm[0, 1],dcm[0,0], -dcm[0, 2],dcm[1, 2], dcm[2, 2])
    return (r1, r2, r3)

In [13]:
Alignment = 60000
Lat = GPS[Alignment,0]
Lon = GPS[Alignment,1]
Alt = GPS[Alignment,2]
W_NUE = GPS[Alignment,3:6]
W_NUE_old = W_NUE
Lat_old = Lat
Alt_old = Alt
Roll = 0
Pitch = 0
Heading = 0
C_B_N = np.eye(3)
NavState = np.zeros(26)
MS_mean_count = 0
gyros_mean_count = 0
accels_mean = np.zeros(3)
gyros_mean = np.zeros(3)
MS_mean = np.zeros(3)

In [14]:
for i in range(Alignment):
    accels_mean = accels_mean + acc[i, :]
    gyros_mean = gyros_mean + gyro[i, :]
    gyros_mean_count = gyros_mean_count + 1
    if (StatusMS[i] == 2):
        MS_mean = MS_mean + MagnSens[i]
        MS_mean_count = MS_mean_count + 1

In [15]:
accels_mean = (accels_mean/gyros_mean_count) / dt # Осреднение показаний акселерометров. Используется счетчик данных гироскопа
gyros_mean = (gyros_mean/gyros_mean_count) / dt # Осреднение показаний гироскопов + пересчет в рад/сек
MS_mean = (MS_mean / MS_mean_count)
# если датчики - ВОГ, то проводится определение курса методом гирокомпасирования
Rbn = CalcDCMnue_align([Lat,Lon,Alt],accels_mean,gyros_mean)
(Heading, Pitch, Roll) = EulerFromDCMnue(Rbn)
C_B_N = DCM_bn(Heading,Pitch,Roll) # Расчет DCM
Heading = twoPiBound(Heading)  # Ограничение
Sensors = np.zeros((6,),dtype = np.float64)
Q = quat(Heading, Roll, Pitch)

In [16]:
print("Параметры после выставки")
print("Курс ")
print(Heading * (180 / np.pi))
print("Тангаж ")
print(Pitch * (180 / np.pi))
print("Крен ")
print(Roll * (180 / np.pi))
print("Широта ")
print(Lat * (180 / np.pi))
print("Долгота ")
print(Lon * (180 / np.pi))
print("Высота ")
print(Alt)
print("W_N ")
print(W_NUE[0])
print("W_U ")
print(W_NUE[1])
print("W_E ")
print(W_NUE[2])

Параметры после выставки
Курс 
333.2617518321868
Тангаж 
6.023054025873441
Крен 
-0.8737296834068249
Широта 
54.9797978
Долгота 
37.663690100000004
Высота 
192.796
W_N 
-0.054999999701976776
W_U 
-0.0430000014603138
W_E 
0.08799999952316284


In [24]:
acc = acc[:,[2,0,1]]
gyro= gyro[:,[2,0,1]]

In [18]:
C_B_N = torch.Tensor(C_B_N)
acc = torch.Tensor(acc)
gyro = torch.Tensor(gyro)

In [25]:
acc[100,:]

array([0.0014615, 0.0014615, 0.0014615])

In [20]:
def SINS(Q : torch.Tensor, acc : torch.Tensor, gyro : torch.Tensor):
    pass